In [ ]:
# default_exp core

# Core functions

> API details.

In [ ]:
# hide
from nbdev.showdoc import *
from fastcore.test import *

In [ ]:
#export
from fastcore.all import *
import pandas as pd

/home/tako/dev/env37/lib/python3.7/site-packages/pandas/compat/__init__.py:85: UserWarning: Could not import the lzma module. Your installed Python is incomplete. Attempting to use lzma compression will result in a RuntimeError.
  warnings.warn(msg)


# Preprocces functions

## M4

In [ ]:
# export
def ts_lists(ts:np.ndarray)-> L:
    """From a `ndarray` of shape (timeseries, max_time) to a list of timeseries with shape (1,time).
    
    max_time = the lenght of the longest timeserie
    time = the lenght of the non-nan values of that specific timeserie    
    """
    lst = L()
    for time_series in ts:
        lst.append(time_series[~np.isnan(time_series)][None,:])
    return lst

In [ ]:
ts = np.array([np.arange(0,10.),
               np.arange(0,10.)
              ])
ts[1,-1] = np.NaN

test_eq(ts_lists(ts)[0],np.arange(0,10.)[None,:])
test_eq(ts_lists(ts)[1],np.arange(0,9.)[None,:])
test_eq(ts_lists(ts)[0].shape,(1,10))

## Gas-usage


In [ ]:
# export
def melted_ts_2_lists(ts:pd.DataFrame, split_on:str)->L:
    return ts

In [ ]:
df = pd.DataFrame({'dt':np.arange(0,100),
                  'x':L([1]*50)+L([2]*50),
                  })

test_eq(type(melted_ts_2_lists(df)),L)
test_eq(len(melted_ts_2_lists(df)),2)

AssertionError: ==:
<class 'pandas.core.frame.DataFrame'>
<class 'fastcore.foundation.L'>

# Fin

In [ ]:
# hide
from nbdev.export import *

notebook2script()

Converted 00_core.ipynb.
Converted 01_data.external.ipynb.
This cell doesn't have an export destination and was ignored:
 
Converted 02_deep4cast_m4_example.ipynb.
Converted 03_data.load.ipynb.
Converted 04_data.transforms.ipynb.
Converted 05_models.wavenet.ipynb.
Converted index.ipynb.
